In [0]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

### Data processing

This is the data for all airplane crashes since 1908 until 2009.

In [3]:
crash1908 = pd.read_csv('https://gist.githubusercontent.com/myroslavarm/ce258616fa3e5da89c45d36ace0c45ae/raw/e4e93ca296a895dbd2145796d6751b90b40372a9/crashdata_since_1908.csv')
crash1908.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [4]:
crash1908.shape

(5268, 13)

Extracting separate year/month columns, used further in some visualisations.

In [0]:
year = pd.DatetimeIndex(crash1908['Date']).year
crash1908 = crash1908.assign(Year=year.values)

month = pd.DatetimeIndex(crash1908['Date']).month
crash1908 = crash1908.assign(Month=month.values)

Only taking the data for 50 years, since 1959 to 2009.

In [6]:
crash = crash1908[crash1908['Year'].astype(int) >= 1959]
crash.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Year,Month
1117,07/15/1966,NaN,Pakistan,Military - Pakistan Air Force,NaN,NaN,Lockheed C-130B Hercules,24142,3768,10.0,10.0,0.0,Struck a mountain.,1966,7
1237,02/13/1970,NaN,"Pin-la-Garenne, France",Avia-Taxi,NaN,NaN,Cessna 402,F-BPJA,402-0100,10.0,10.0,0.0,NaN,1970,2
1577,01/13/1962,NaN,"Near Tippi, Ethiopia",Ethiopian Airlines,NaN,Tippi - Jimma,Douglas DC-3,ET-T-1,10053,18.0,5.0,1.0,Swerved off the runway while taking off and st...,1962,1
1578,01/06/1959,11:20,"Mt. Pena Blanca, Guatemala",Sahsa Airlines,NaN,Copan - Nuovo Octopeque,Douglas DC-3,XH-SAA,19667,5.0,5.0,0.0,Crashed into Mt. Pena Blanca while en route. T...,1959,1
1579,01/08/1959,20:32,"Mt. Holston, Tennessee",Southeast Airlines,308,Nashville - Knoxville - Tri-City Aport,Douglas DC-3,N18941,2007,10.0,10.0,0.0,The aircraft struck a mountain during an ILS a...,1959,1


Excluding all military vehicle crashes.

In [7]:
crash = crash[~crash['Operator'].str.contains("Military|military", na=False)]
crash.shape

(3173, 15)

Getting days of the week from dates.

In [8]:
crash['Date'] = pd.to_datetime(crash['Date'])
crash['Weekday'] = crash['Date'].dt.day_name()
crash.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Year,Month,Weekday
1237,1970-02-13,NaN,"Pin-la-Garenne, France",Avia-Taxi,NaN,NaN,Cessna 402,F-BPJA,402-0100,10.0,10.0,0.0,NaN,1970,2,Friday
1577,1962-01-13,NaN,"Near Tippi, Ethiopia",Ethiopian Airlines,NaN,Tippi - Jimma,Douglas DC-3,ET-T-1,10053,18.0,5.0,1.0,Swerved off the runway while taking off and st...,1962,1,Saturday
1578,1959-01-06,11:20,"Mt. Pena Blanca, Guatemala",Sahsa Airlines,NaN,Copan - Nuovo Octopeque,Douglas DC-3,XH-SAA,19667,5.0,5.0,0.0,Crashed into Mt. Pena Blanca while en route. T...,1959,1,Tuesday
1579,1959-01-08,20:32,"Mt. Holston, Tennessee",Southeast Airlines,308,Nashville - Knoxville - Tri-City Aport,Douglas DC-3,N18941,2007,10.0,10.0,0.0,The aircraft struck a mountain during an ILS a...,1959,1,Thursday
1580,1959-01-11,NaN,"Rio de Janeiro, Brazil",Lufthansa,NaN,Hamburg - Rio de Janeiro,Lockheed 1049G Super Constellation,D-ALAK,4602,39.0,36.0,0.0,The plane descended over Guanabara Bay in rain...,1959,1,Sunday


### Seeing which months and days of the week had the largest amounts of plane crashes in each year, as well as each hour of each day of the week

In [9]:
vis1 = crash.groupby(["Date", "Time", "Weekday", "Year", "Month", "Aboard", "Fatalities"])['Ground'].count().reset_index()
vis1.head()

,Date,Time,Weekday,Year,Month,Aboard,Fatalities,Ground
0,1959-01-06,11:20,Tuesday,1959,1,5.0,5.0,1
1,1959-01-08,20:32,Thursday,1959,1,10.0,10.0,1
2,1959-01-16,21:40,Friday,1959,1,52.0,51.0,1
3,1959-01-22,15:00,Thursday,1959,1,15.0,10.0,1
4,1959-01-27,06:10,Tuesday,1959,1,6.0,2.0,1


Manually getting rid of several inconsistensies in the way time is written.

In [0]:
vis1['Time'] = vis1.Time.str.replace("c: |c:",'')
vis1['Time'] = vis1.Time.str.replace("'",':')
vis1['Time'] = vis1.Time.str.replace(".",':')
vis1['Time'] = vis1.Time.str.replace("0943",'09:43')
vis1['Time'] = vis1.Time.str.replace("114:20",'14:20')

Converting to the proper format.

In [0]:
vis1['Time'] = pd.to_datetime(vis1.Time, format='%H:%M')

Visualising.

In [12]:
alt.Chart(vis1).mark_circle().encode(
    x = alt.X('year(Date):T', scale = alt.Scale(zero = False, domain = (1958, 2010)), title = "Year"),
    y = alt.Y('Month:Q', title = "Month"),

    color = alt.value('red'),
    size = alt.Size('Fatalities:Q'),
    tooltip = ['Aboard', 'Fatalities']
).properties(width = 1000, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Number of fatalities in each month of each year',
                    font = 'Inconsolata', fontSize = 20, color = '#3E454F'))

alt.Chart(...)

In [13]:
vis12 = vis1.groupby(["Year", "Weekday", "Aboard", "Fatalities", "Date", "Time"]).sum().reset_index()
vis12.head()

,Year,Weekday,Aboard,Fatalities,Date,Time,Month,Ground
0,1959,Friday,3.0,2.0,1959-08-14,1900-01-01 17:14:00,8,1
1,1959,Friday,18.0,1.0,1959-03-13,1900-01-01 08:00:00,3,1
2,1959,Friday,27.0,26.0,1959-10-30,1900-01-01 20:40:00,10,1
3,1959,Friday,52.0,51.0,1959-01-16,1900-01-01 21:40:00,1,1
4,1959,Friday,68.0,68.0,1959-06-26,1900-01-01 17:35:00,6,1


In [14]:
alt.Chart(vis12).mark_circle().encode(
    x = alt.X('year(Date):T', scale = alt.Scale(zero = False, domain = (1958, 2010)), title = "Year"),
    y = alt.Y('Weekday:N', title = "Weekday", type = 'ordinal', sort = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),

    color = alt.value('red'),
    size = alt.Size('Fatalities:Q'),
    tooltip = ['Aboard', 'Fatalities']
).properties(width = 1000, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Number of fatalities in each day of the week of each year',
                    font = 'Inconsolata', fontSize = 20, color = '#3E454F'))

alt.Chart(...)

In [15]:
slider = alt.binding_range(min=1959, max=2009, step=1)
select_year = alt.selection_single(name="year", fields=['Year'], bind=slider, init={'Year': 1959})

alt.Chart(vis12).mark_circle().encode(
    x = alt.X('Weekday:N', title = "Weekday", type = 'ordinal', sort = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),
    y = alt.Y('Time:T', timeUnit = 'hours', title = "Time of day", scale=alt.Scale(padding=20)),
    color = alt.value('red'),
    size = alt.Size('Fatalities:Q'),
    tooltip = ['Aboard', 'Fatalities']
).properties(width = 300, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Times of day in each day of the week with most fatalities',
                    font = 'Inconsolata', fontSize = 20, color = '#3E454F')).add_selection(select_year).transform_filter(select_year)

alt.Chart(...)

Conclusion based on visualisations: very early morning times had less crashes anf fatalities, which also could be attributed to there being fewer flights at that time. However, it seems that a large amount of plane crashes happens in the evening.

### Number of crashes per year

Grouping by number of crashes, people aboard/fatalities per year.

In [16]:
df1 = crash.groupby(["Year"])["Aboard"].sum().reset_index(name='Aboard_Per_Year')
df2 = crash.groupby(["Year"])["Fatalities"].sum().reset_index(name='Fatalities_Per_Year')
vis2 = crash.groupby(["Year"]).size().reset_index(name='No_of_Crashes')
vis2 = vis2.assign(Aboard_Per_Year=df1['Aboard_Per_Year'].values)
vis2 = vis2.assign(Fatalities_Per_Year=df2['Fatalities_Per_Year'].values)
vis2['Year'] = pd.to_datetime(vis2.Year, format='%Y')
vis2.head()

,Year,No_of_Crashes,Aboard_Per_Year,Fatalities_Per_Year
0,1959-01-01,59,1248.0,1027.0
1,1960-01-01,51,1636.0,1294.0
2,1961-01-01,44,1638.0,1273.0
3,1962-01-01,63,2212.0,1828.0
4,1963-01-01,48,1321.0,1097.0


In [17]:
alt.Chart(vis2).mark_bar().encode(
    x = alt.X('year(Year):T', scale = alt.Scale(zero = False, domain=(1959, 2010)), title="Year"),
    y = alt.Y('No_of_Crashes:Q', scale = alt.Scale(zero = False, domain=(0, 100)), title="Number of crashes"),
    tooltip = ['year(Year)','No_of_Crashes:Q']
).properties(width = 1100, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Plane crashes in 50 years', subtitle = ['1959-2009', ' '],
                    font = 'Inconsolata', fontSize = 20, color = '#3E454F', subtitleFont = 'Inconsolata', subtitleFontSize = 18, subtitleColor = '#3E454F'))

alt.Chart(...)

Conclusion based on the visualisation: considering that the amount of flights performed has been steadily rising throughout this half-century, it can also be inferred that flying has become much safer than it used to be.
P.S. a little lowering of number of plane crashes in the early 1980s is also very interesting.

### Number of people aboard / fatalities per year and per month

In [18]:
vis23 = vis2.drop('No_of_Crashes', 1)
vis23 = vis23.melt('Year', var_name='Measurement', value_name='Values')
vis23 = vis23.groupby(["Year", "Measurement", "Values"]).sum().reset_index()
vis23.head()

,Year,Measurement,Values
0,1959-01-01,Aboard_Per_Year,1248.0
1,1959-01-01,Fatalities_Per_Year,1027.0
2,1960-01-01,Aboard_Per_Year,1636.0
3,1960-01-01,Fatalities_Per_Year,1294.0
4,1961-01-01,Aboard_Per_Year,1638.0


Comparing number of people aboard to the number of people who died in plane crashes.

In [19]:
bars1 = alt.Chart(vis2).mark_bar(color = 'magenta').encode(
    x = alt.X('year(Year):T', scale = alt.Scale(zero = False, domain = (1959, 2010)), title = "Year"),
    y = alt.Y('Aboard_Per_Year:Q', title = "Number of people aboard / fatalities", scale = alt.Scale(zero = False, domain = (0, 4000))),
    tooltip = ['year(Year)', 'Aboard_Per_Year']
)

bars2 = alt.Chart(vis2).mark_bar(color = 'purple').encode(
    x = alt.X('year(Year):T', scale = alt.Scale(zero = False, domain = (1959, 2010))),
    y = alt.Y('Fatalities_Per_Year:Q', scale = alt.Scale(zero = False, domain = (0, 4000))),
    tooltip = ['year(Year)', 'Fatalities_Per_Year']
)

(bars1 + bars2).properties(width = 1100, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Comparison of the total amount of people on board vs fatalities',
                    subtitle = ['version 1', ' '], font = 'Inconsolata', fontSize = 20, color = '#3E454F',
                    subtitleFont = 'Inconsolata', subtitleFontSize = 18, subtitleColor = '#3E454F'))

alt.LayerChart(...)

In [20]:
alt.Chart(vis23).mark_line().encode(
    x = alt.X('year(Year):T', scale = alt.Scale(zero = False, domain = (1959, 2010)), title = "Year"),
    y = alt.Y('Values:Q', title = "Number of people aboard / fatalities", scale = alt.Scale(zero = False, domain = (0, 4000))),
    color = ('Measurement:N')
).properties(width = 1000, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Comparison of the total amount of people on board vs fatalities',
                    subtitle = ['version 2', ' '], font = 'Inconsolata', fontSize = 20, color = '#3E454F',
                    subtitleFont = 'Inconsolata', subtitleFontSize = 18, subtitleColor = '#3E454F'))
# here you have the legend but on the whole the visualisation isn't as straightfoward as the barchart above

alt.Chart(...)

Conclusion based on the visualisation: on average, most people die in plane crashes; there are crashes were fatality rate is 100% and there are crashes were a lot of people survive, but as we see on a yearly basis, on average far more than half the people in plane crashes die.

Grouping by month.

In [0]:
df3 = crash.groupby(["Year","Month"])["Aboard"].sum().reset_index(name='Aboard_Per_Month')
df4 = crash.groupby(["Year","Month"])["Fatalities"].sum().reset_index(name='Fatalities_Per_Month')
vis3 = crash.groupby(["Year","Month"]).size().reset_index(name='No_of_Crashes')
vis3 = vis3.assign(Aboard_Per_Month=df3['Aboard_Per_Month'].values)
vis3 = vis3.assign(Fatalities_Per_Month=df4['Fatalities_Per_Month'].values)
vis3['Month'] = pd.to_datetime(vis3.Month, format='%m')

Visualising.

In [22]:
slider = alt.binding_range(min=1959, max=2009, step=1)
select_year = alt.selection_single(name="year", fields=['Year'], bind=slider, init={'Year': 1959})

bars1 = alt.Chart(vis3).mark_bar().encode(
    x = alt.X('month(Month):T', title = "Month"),
    y = alt.Y('Aboard_Per_Month:Q', scale = alt.Scale(domain = [0, 800]), title = "Number of people aboard / fatalities"),
    color = alt.value('magenta'))

bars2 = alt.Chart(vis3).mark_bar().encode(
    x = alt.X('month(Month):T'),
    y = alt.Y('Fatalities_Per_Month:Q', scale = alt.Scale(domain = [0, 1000])),
    color = alt.value('purple'))

(bars1 + bars2).properties(width = 500, background = '#F0F1F1', padding = 30, title = alt.TitleParams(text = 'Comparison of the total amount of people on board vs fatalities',
                    font = 'Inconsolata', fontSize = 20, color = '#3E454F')).add_selection(select_year).transform_filter(select_year)

alt.LayerChart(...)

Conclusion based on the visualisation: it seems that even though there are less plane crashes than used to be before (see visualisations in previous sections), that hasn't exactly changed the situation when the plane crashes do happen -- over the years they haven't become less deadly. Sure, over the years you can spot than in some instances more people survived than in others, but as a whole 50 years of commercial flight & aviation history haven't seemed to change the fact that when the plane crashes, usully, if not all, most people still die.

In conclusion, this research project visually demonstrates that even though the practices over the years have gotten better and the amount of plane crashes has decreased, humanity has yet to figure out how to avoid plane crashes being deadly when they do happen.